# FIT5196 Assessment 2 Task 2
#### Student Name: Vipul Krishnan Muralee Dharan
#### Student ID: 28104641

Date: 11/05/2018

Version: 1.0

Environment: Python 3.6.1 and Anaconda 4.3.21 (64-bit)

Libraries used:
* pandas v0.20.1 (for dataframe operations)
* fuzzywuzzy v0.16.0 (for string comparison)
* numpy v1.12.1(for mathematical operations)
* rendom (python module, comes with python)(for generating random numbers)

## 1. Introduction
Task 2 includes 
* correcting schema conflicts, following the schema of dataset1
* implement the semantic mapping and merge between dataset1 and datset2 to produce one unified table
* detect and resolve duplication in the unified table.
* find a proper global key for the integrated job data and explain.


## 2. Installing and Importing Libraries

We need the fuzzywuzzy library for this task. We use it in various places for string comparison. Fuzzywuzzy uses another library 'python-Levenshtein' for speeding up its processes (SeatGeek, 2018). If these libraries are not istalled, please install them using the below code and then import all libraries. 

In [1]:
!pip install python-Levenshtein
!pip install fuzzywuzzy

You are using pip version 9.0.2, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.2, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
from fuzzywuzzy import fuzz
from random import randint
import numpy as np

## 3. Loading and Examining Data
The first step is to load the data from the CSV file.

In [3]:
# loading the data as pandas dataframe
df1 = pd.read_csv("dataset1_solution.csv")
df2 = pd.read_csv("dataset2_integration.csv")

# checking first 5 rows to see the structure of the data
df2.head()

,Id,Source Name,Title,location,Contract Type,Contract Time,Company,Category,Salary per month,OpenDate,CloseDate
0,69247680,jobs.guardian.co.uk,Business Development Exec research / insight ...,London,ft,perm.,BOYCE RECRUITMENT,"PR, Advertising & Marketing Jobs",2125,2012-03-01 15:00:00,2012-03-31 15:00:00
1,69247682,icaewjobs.com,Audit Senior London,London,ft,contr.,Pro Finance,Finance & Accounting Jobs,3600,2013-03-14 12:00:00,2013-04-13 12:00:00
2,69247685,jobs.guardian.co.uk,PR & Social Media Account Executive Top Inter...,Central London,ft,perm.,ECOM RECRUITMENT LTD,"PR, Advertising & Marketing Jobs",1917,2012-02-14 12:00:00,2012-03-15 12:00:00
3,69247688,jobs.guardian.co.uk,Content Manager,London,ft,perm.,NAKAMA LONDON,"PR, Advertising & Marketing Jobs",2083,2012-10-21 00:00:00,2012-12-20 00:00:00
4,69247694,jobs.guardian.co.uk,AV PRODUCTION MANAGER,Worcestershire,ft,perm.,LIVE RECRUITMENT,"PR, Advertising & Marketing Jobs",2292,2012-02-22 00:00:00,2012-04-22 00:00:00


In [4]:
df1.head()

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
0,12612628,Engineering Systems Analyst,Dorking,non-specified,permanent,Gregory Martin International,Engineering Jobs,24996.0,cv-library.co.uk,20121103T000000,20121203T000000
1,12612830,Stress Engineer Glasgow,Glasgow,non-specified,permanent,Gregory Martin International,Engineering Jobs,30000.0,cv-library.co.uk,20130108T150000,20130408T150000
2,12612844,Modelling and simulation analyst,Hampshire,non-specified,permanent,Gregory Martin International,Engineering Jobs,30000.0,cv-library.co.uk,20130726T150000,20130924T150000
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Surrey,non-specified,permanent,Gregory Martin International,Engineering Jobs,27504.0,cv-library.co.uk,20121214T000000,20130314T000000
4,12613647,"Pioneer, Miser Engineering Systems Analyst",Surrey,non-specified,permanent,Gregory Martin International,Engineering Jobs,24996.0,cv-library.co.uk,20131025T000000,20131224T000000


## 4. Resolving schema conflicts and merging data
Now we check for the schema conflicts.

### 4a. Detecting Schema Conflicts
We compare each column of df1 to curresponding column of schema 2 to find schema conflicts. We use different methods for this. We can compare data types of the values in the column and also use the semantics of the name of the attributes. 

Also we can compare the data itself. We can do this by comparing the df1 data itself for a number of times and obtainng the similarity value and then comparing the df1 values with df2 values. The the two averaged similarity values are then compared. If they are similar, then the columns in df1 and df2 have similar data.

As we can see, the number of columns are equal in both dataframes.

In [5]:
schemanamematch = []

# comparing the column names of each columns
for i in range(len(df1.columns)):
    schemanamematch.append(fuzz.ratio(df1.columns[i], df2.columns[i]))

print(schemanamematch)

[100, 12, 15, 30, 80, 30, 40, 25, 31, 100, 100]


In [6]:
schemadtmatch = []

# comparing the data types of each columns
for i in range(len(df1.columns)):
    schemadtmatch.append(1 if df1.dtypes[i] == df2.dtypes[i] else 0)

print(schemadtmatch)

[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1]


In [7]:
datamatch = []

# checking data match
for c in range(len(list(df1.columns))):
    avlist = []
    # obtains the avg fuzz ratio value after comparing random selected values in the df1
    for i in range(10000):
        avlist.append(fuzz.ratio(str(df1.iloc[randint(0,len(df1) - 1), c]), str(df1.iloc[randint(0,len(df1) - 1), c])))
    df1avg = np.mean(avlist)
    crossavlist = []
    
    # obtains the avg fuzz ratio value after comparing random selected values in the df1 and df2
    for j in range(10000):
        crossavlist.append(fuzz.ratio(str(df1.iloc[randint(0,len(df1) - 1), c]), str(df2.iloc[randint(0,len(df1) - 1), c])))
    crossavg = np.mean(crossavlist)
    
    #finding the differnece between avg values
    datamatch.append(abs(df1avg - crossavg))
    
print(datamatch)

[7.8111000000000033, 10.9941, 6.0846999999999998, 55.48790000000001, 46.130399999999995, 14.461400000000001, 31.839000000000002, 55.3367, 50.525999999999996, 13.10710000000001, 12.740400000000001]


As we see, the first result says the field names matches for column 0,9 and 10 only. The second method says, the data types of the column matches for all except column 7 and 8.

The third method says there are significant data dissimilarities in columns 3,4,6,7,8 and possible conflicts in 1,

A combined analysis shows that there are some schema conflicts in all columns except first one.


### 4b. Semantic mapping and merge between dataset1 and datset2

This part includes mapping of each columns in df1 to curresponding columns in df2. 

We can use matchers for matching the attributes. In general we can use a number of matchers and use a combiner to increase the accuracy. But as our columns are properly named, a matcher which detects the similarities in column names would be enough for us.

The below code creates a similarity matrix based on column names

In [8]:
# similarity matrix
match = []

# for each column in df1
for i in df1.columns:
    row = []
    # for each column name in df2
    for j in df2.columns:
        # compare the names
        row.append(fuzz.ratio(i,j))
    # add the values in similarity matrix
    match.append(row)
    
match

[[100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 12, 100, 15, 22, 33, 0, 31, 19, 31, 29],
 [0, 32, 15, 88, 29, 38, 40, 38, 25, 25, 35],
 [0, 35, 24, 30, 96, 80, 42, 40, 29, 40, 48],
 [0, 43, 35, 40, 80, 96, 32, 40, 21, 40, 48],
 [0, 22, 0, 40, 40, 30, 100, 40, 17, 27, 38],
 [0, 21, 31, 38, 38, 38, 40, 100, 25, 38, 47],
 [0, 44, 19, 25, 28, 21, 26, 25, 75, 25, 24],
 [0, 95, 13, 33, 35, 43, 24, 22, 31, 33, 42],
 [0, 32, 31, 25, 38, 38, 27, 38, 33, 100, 59],
 [0, 40, 29, 47, 45, 45, 38, 47, 24, 59, 100]]

Now we write a code to obtain the most probable order of columns in df2 to correctly join with df1

In [9]:
# list for new order
reorder = []
# for each row, finding the position of max value and adding the index to array
for i in range(len(match)):
    reorder.append(match[i].index(max(match[i])))
reorder

[0, 2, 3, 4, 5, 6, 7, 8, 1, 9, 10]

Now we reorder the columns using this

In [10]:
# changing the colum indices to numbers
df2.columns = list(range(11))
# changing the column order
df2 = df2[reorder]
# readding the column names
df2.columns = df1.columns

Now the column order and names are uniform, But we need to check the data values are matching. For example, for the columns, ContractTime, ContractType and Category, the number of unique values are limited. But a look at both deataframe heads shows that these values are labled in different style. We can use semantic mapping to correct these error too. For example, if full-time df1 is labled as ft df2, we have to map ft to full-time. We use the below code for that. It also prints the mapping below as dictionaries..

In [11]:
# for each columns in specified list
for k in ['ContractType', 'ContractTime', 'Category']:
    
    # finds the unique values in df1 and df2
    df1ct = list(df1[k].unique())
    df2ct = list(df2[k].unique())
    
    # match matrix
    match1 = []
    
    # comparing each values from df1 with that of df2
    for i in df1ct:
        row = []
        for j in df2ct:
            # fuzzy is used for comparison along with a weight for same first letter
            row.append(fuzz.ratio(i,j) + (15 if (i[0] == j[0]) else 0))
        match1.append(row)
    
    repl = {}
    # each value in df2 is added as key and the value in df1 which maximize the match is added as value of the key
    for i in range(len(match1)):
        repl[df2ct[match1[i].index(max(match1[i]))]] = df1ct[i]
    
    # printing the dictionay
    print(repl)
    
    # replacing the values in df2
    df2[k].replace(repl, inplace = True)

{'n/a': 'non-specified', 'ft': 'full-time', 'pt': 'part-time'}
{'perm.': 'permanent', 'n/a': 'non-specified', 'contr.': 'contract'}
{'Engineering Jobs': 'Engineering Jobs', 'Finance & Accounting Jobs': 'Accounting & Finance Jobs', 'Healthcare & Nursing Jobs': 'Healthcare & Nursing Jobs', 'Hospitality & Catering Jobs': 'Hospitality & Catering Jobs', 'IT Jobs': 'IT Jobs', 'Sales Jobs': 'Sales Jobs', 'Teaching Jobs': 'Teaching Jobs', 'PR, Advertising & Marketing Jobs': 'PR, Advertising & Marketing Jobs'}


In [12]:
df2

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
0,69247680,Business Development Exec research / insight ...,London,full-time,permanent,BOYCE RECRUITMENT,"PR, Advertising & Marketing Jobs",2125,jobs.guardian.co.uk,2012-03-01 15:00:00,2012-03-31 15:00:00
1,69247682,Audit Senior London,London,full-time,contract,Pro Finance,Accounting & Finance Jobs,3600,icaewjobs.com,2013-03-14 12:00:00,2013-04-13 12:00:00
2,69247685,PR & Social Media Account Executive Top Inter...,Central London,full-time,permanent,ECOM RECRUITMENT LTD,"PR, Advertising & Marketing Jobs",1917,jobs.guardian.co.uk,2012-02-14 12:00:00,2012-03-15 12:00:00
3,69247688,Content Manager,London,full-time,permanent,NAKAMA LONDON,"PR, Advertising & Marketing Jobs",2083,jobs.guardian.co.uk,2012-10-21 00:00:00,2012-12-20 00:00:00
4,69247694,AV PRODUCTION MANAGER,Worcestershire,full-time,permanent,LIVE RECRUITMENT,"PR, Advertising & Marketing Jobs",2292,jobs.guardian.co.uk,2012-02-22 00:00:00,2012-04-22 00:00:00
5,69247712,Search Account Manager/Director (to go ClientS...,Central London,full-time,permanent,TANDEM RECRUITMENT LTD,"PR, Advertising & Marketing Jobs",4583,jobs.guardian.co.uk,2013-04-15 12:00:00,2013-07-14 12:00:00
6,69247714,Account Director – Oxford Salary up to **** d...,Oxfordshire,full-time,permanent,ALLEN ASSOCIATES,"PR, Advertising & Marketing Jobs",4375,jobs.guardian.co.uk,2012-04-01 15:00:00,2012-05-01 15:00:00
7,69247718,Marketing Executive **** month FTC North Oxf...,Oxfordshire,full-time,contract,ALLEN ASSOCIATES,"PR, Advertising & Marketing Jobs",2417,jobs.guardian.co.uk,2013-03-29 00:00:00,2013-06-27 00:00:00
8,69247722,Senior Search Executive Up to ****k Top inde...,London,full-time,permanent,ECOM RECRUITMENT LTD,"PR, Advertising & Marketing Jobs",1833,jobs.guardian.co.uk,2012-08-25 15:00:00,2012-10-24 15:00:00
9,69247727,Portfolio Manager Credit Card,Wiltshire,full-time,permanent,NATIONWIDE BUILDING SOCIETY,"PR, Advertising & Marketing Jobs",3708,jobs.guardian.co.uk,2013-04-04 15:00:00,2013-06-03 15:00:00


The other issues in the schema are date format conflicts and the repesentation of salary/month in df2 vs salary/year in df1. We solve these now.

In [13]:
# solving date format conflicts
df2.loc[:,'OpenDate'] = df2.OpenDate.apply(lambda x:x.replace('-','').replace(':','').replace(' ','T')) 
df2.loc[:,'CloseDate'] = df2.CloseDate.apply(lambda x:x.replace('-','').replace(':','').replace(' ','T')) 

In [14]:
# multiplying salary per month by 12 to approximate salary per annum
df2.loc[:,'Salary per annum'] = df2['Salary per annum'].apply(lambda x: x*12)

Now we can join the dataframes, and reset the column index.

In [15]:
# concatenating the dataframes
df = pd.concat([df1, df2])

In [16]:
# resetting row indices
df.reset_index()

,index,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
0,0,12612628,Engineering Systems Analyst,Dorking,non-specified,permanent,Gregory Martin International,Engineering Jobs,24996.0,cv-library.co.uk,20121103T000000,20121203T000000
1,1,12612830,Stress Engineer Glasgow,Glasgow,non-specified,permanent,Gregory Martin International,Engineering Jobs,30000.0,cv-library.co.uk,20130108T150000,20130408T150000
2,2,12612844,Modelling and simulation analyst,Hampshire,non-specified,permanent,Gregory Martin International,Engineering Jobs,30000.0,cv-library.co.uk,20130726T150000,20130924T150000
3,3,12613049,Engineering Systems Analyst / Mathematical Mod...,Surrey,non-specified,permanent,Gregory Martin International,Engineering Jobs,27504.0,cv-library.co.uk,20121214T000000,20130314T000000
4,4,12613647,"Pioneer, Miser Engineering Systems Analyst",Surrey,non-specified,permanent,Gregory Martin International,Engineering Jobs,24996.0,cv-library.co.uk,20131025T000000,20131224T000000
5,5,19047429,Trainee Mortgage Advisor East Midlands,East Midlands,non-specified,permanent,Brite Recruitment,Accounting & Finance Jobs,21000.0,cv-library.co.uk,20120724T000000,20120922T000000
6,6,20199757,"PROJECT ENGINEER, PHARMACEUTICAL",Witney,non-specified,permanent,Matchbox Recruiting Ltd,Healthcare & Nursing Jobs,37500.0,cv-library.co.uk,20130328T120000,20130427T120000
7,7,20797143,Chef de Partie Award Winning Restaurant Exce...,Derby,non-specified,non-specified,Chef Results,Hospitality & Catering Jobs,15996.0,caterer.com,20131027T000000,20131226T000000
8,8,22579462,Quality Engineer,Gateshead,non-specified,permanent,Asset Appointments,Engineering Jobs,21996.0,cv-library.co.uk,20130815T000000,20130914T000000
9,9,22933091,Chef de Partie Award Winning Dining Live In ...,UK,non-specified,non-specified,Chef Results,Hospitality & Catering Jobs,18000.0,caterer.com,20120814T000000,20120828T000000


### 4c. Detecting and Resolving Duplication in the Unified Table
The duplication detection can be done using the function df.duplicated(). We try using this function

In [17]:
# checking for duplicates. 
df[df.duplicated()]

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate


This suggests there are no duplicate rows with all the fields as duplicates, but still there can be some attributes as duplicates. We try describing the dataframe and finding the number of duplicates in each column.

In [18]:
df.describe(include = ['O'])

,Title,Location,ContractType,ContractTime,Company,Category,SourceName,OpenDate,CloseDate
count,50353,50353,50353,50353,49255,50353,50353,50353,50353
unique,50159,482,3,3,8369,8,95,2193,2413
top,French speaking Oracle Liferay Technician/Engi...,UK,non-specified,permanent,Not Available,IT Jobs,totaljobs.com,20130127T150000,20120908T150000
freq,2,8828,39956,32699,3836,13939,10134,41,40


In [19]:
for c in df.columns:
    print(c + " : " + str(len(df[df[c].duplicated()])))

Id : 0
Title : 194
Location : 49871
ContractType : 50350
ContractTime : 50350
Company : 41983
Category : 50345
Salary per annum : 48132
SourceName : 50258
OpenDate : 48160
CloseDate : 47940


Id is unique and Title has the least number of duplicates. We try removing the Id and search for duplicates.

In [20]:
df[df.duplicated(subset= ['Title', 'Company', 'Location'])]

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
133,69249874,Staff Nurse BexhillonSea,Bexhill-On-Sea,full-time,non-specified,Professional Social Care Solutions,Healthcare & Nursing Jobs,25500.0,staffnurse.com,20130220T150000,20130322T150000
166,69250133,"Support Care Workers, Luton Bedford (Evenings ...",UK,non-specified,contract,Diamond Resourcing,Healthcare & Nursing Jobs,17280.0,cv-library.co.uk,20130525T120000,20130724T120000
1084,69541267,TRAINEE IT MANAGER SYSTEMS ADMIN ****k,Cliddesden,non-specified,permanent,Circle Recruitment,IT Jobs,32496.0,jobserve.com,20120206T000000,20120406T000000
1218,69545085,"Assistant Manager ST ANDREWS, FIFE Restaurant ...",Fife,non-specified,non-specified,Addison Fowle Ltd,Hospitality & Catering Jobs,18996.0,caterer.com,20121203T000000,20121217T000000
1528,69559108,Year **** Class Teacher Urgently Required for ...,West End,non-specified,contract,Academics Ltd,Teaching Jobs,32400.0,cv-library.co.uk,20121009T150000,20121208T150000
1671,69570901,German Speaker,Brighton,non-specified,contract,Page Personnel Finance,Accounting & Finance Jobs,8640.0,totaljobs.com,20131130T000000,20131230T000000
2071,69593348,ConsultantLondon Markets Insurance,Central London,non-specified,permanent,Xceed,IT Jobs,50004.0,jobserve.com,20130818T150000,20130917T150000
2143,69596435,Maintenance Engineer Electrical,Leeds,non-specified,permanent,Clare Moore,Engineering Jobs,30744.0,jobsite.co.uk,20131030T150000,20140128T150000
2378,69614996,Clinical Nurse Trainer Renal Dialysis/ITU,North West London,non-specified,permanent,Kirkham Young Ltd,Healthcare & Nursing Jobs,38004.0,cv-library.co.uk,20130213T150000,20130315T150000
2512,69621828,Assistant Manager Carluccio's Aberdeen New ...,Aberdeen,non-specified,non-specified,Carluccio's,Hospitality & Catering Jobs,24996.0,caterer.com,20130928T000000,20131028T000000


In [21]:
# dropping the duplicates while keeping the first occurrance
df.drop_duplicates(subset= ['Title', 'Company', 'Location'], keep='first', inplace = True)

Starting with all attributes except the Id, and eliminating one by one if there is no change in the number of duplicates detected, we could reduce untill the subset becomes only ['Title', 'Company', 'Location']. 

So this is the global key for our datasets for finding the duplicates. Adding any other attributes along with this makes no change, how ever dropping any of ['Title', 'Company', 'Location'] in the duplicated() function makes it find more duplicates.

Also please note that some uniqueness of each of this variables are caused by the clean up we have done in the previous task. An analysis shows if we use the dataset df1 with error for finding the duplicates, our global key can be reduces to only the 'Title' and we will get 200 duplicates.

## 5. Storing Data in File

Now we store the combined data to the csv file

In [23]:
# storing data into file
df.to_csv("dataset1_dataset2_solution.csv", encoding = 'utf-8', index=False)

## 6. Summary

In this task we combined the data we cleaned up earlier with the extra data given. 

As a first step we checked schema conflicts. We used semantic methods for mapping columns and finding errors. We corrected the errors one by one.

In the second part, we tested for duplicate rows and find out a global key for the data to find the duplicates.

## References

- Pandas (2018). 'Pandas documentation'. Retrieved from https://pandas.pydata.org
